# Data Cleaning — Goodbooks-10k Top 100 (Updated)
**Objetivo:**  
 
1. Cargar y explorar los ficheros de ratings, metadatos y tags.  
2. Seleccionar los 100 libros más valorados.  
3. Filtrar usuarios con actividad mínima (≥ 20 ratings).  
4. Binarizar las calificaciones (≥ 4 → “like”).  
5. Pivotar a matriz dispersa 0/1 `R_binary`.  
6. Extraer y guardar metadata enriquecida (título, autor, top-tags).



Importamos las librerías

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

Cargamos los datos

In [2]:
ratings = pd.read_csv('./data/ratings.csv')
books = pd.read_csv('./data/books.csv')

Limpieza: eliminamos usuarios y libros con menos de 5 valoraciones

In [3]:
min_ratings = 5
user_counts = ratings['user_id'].value_counts()
book_counts = ratings['book_id'].value_counts()

ratings_clean = ratings[
    ratings['user_id'].isin(user_counts[user_counts >= min_ratings].index) &
    ratings['book_id'].isin(book_counts[book_counts >= min_ratings].index)
].copy()

n_users_clean = ratings_clean['user_id'].nunique()
n_items_clean = ratings_clean['book_id'].nunique()

print(f"Usuarios con ≥{min_ratings} ratings: {n_users_clean}")
print(f"Libros   con ≥{min_ratings} ratings: {n_items_clean}")

Usuarios con ≥5 ratings: 53424
Libros   con ≥5 ratings: 10000


Dividimos en train y test, tomando valoración y usuario

In [4]:
train, test = train_test_split(ratings_clean, test_size=0.2, random_state=42)

Baseline: media global

In [5]:
global_mean = train['rating'].mean()
test['pred_global_mean'] = global_mean

# mean squared error y mean absolute error
rmse_global = np.sqrt(mean_squared_error(test['rating'], test['pred_global_mean']))
mae_global = mean_absolute_error(test['rating'], test['pred_global_mean'])

Baseline: media por usuario

In [6]:
user_mean = train.groupby('user_id')['rating'].mean()
test = test.join(user_mean, on='user_id', rsuffix='_user_mean')
test['pred_user_mean'] = test['rating_user_mean'].fillna(global_mean)

rmse_user = np.sqrt(mean_squared_error(test['rating'], test['pred_user_mean']))
mae_user = mean_absolute_error(test['rating'], test['pred_user_mean'])

Mostramos los resultados

In [ ]:
results = pd.DataFrame({
    'Baseline': ['Global Mean', 'User Mean'],
    'RMSE': [rmse_global, rmse_user],
    'MAE': [mae_global, mae_user]
})

print('Resultados de Baseline\n', results)

Resultados de Baseline
       Baseline      RMSE       MAE
0  Global Mean  0.990329  0.774018
1    User Mean  0.893159  0.700713


RMSE (~1): en promedio, la predicción se equivoca en 1 estrella sobre 5.
MAE (~0.7): el error medio absoluto es  de, más o menos, 0.7 estrellas.

Al pasar de global mean a user mean el RMSE se reduce a 0.89, por lo que ya tenemos una mejora del 10%. Estos datos serán el baseline contra el que compararemos los 3 métodos de filtrado colaborativo para ver si nos son útiles.



Para comenzar con las implementaciones guardaremos en nuestra carpeta data/processed ratings_clean, train y test para mayor orden.

In [8]:
import os
processed_dir = 'data/processed'

if not os.path.isdir(processed_dir):
    os.makedirs(processed_dir, exist_ok=True)
    
ratings_clean.to_csv(os.path.join(processed_dir, 'ratings_clean.csv'), index=False)
train.to_csv(os.path.join(processed_dir, 'train.csv'), index=False)
test.to_csv(os.path.join(processed_dir, 'test.csv'), index=False)

print("Datos guardados en 'data/processed':")
print(os.listdir(processed_dir))

Datos guardados en 'data/processed':
['ratings_clean.csv', 'test.csv', 'train.csv']


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 1) Aseguramos que test_df tiene columna de relevancia binaria
test_df['rel'] = (test_df.rating >= 4).astype(int)

# 2) Verdaderos
y_true    = test_df.rating.values
y_true_bin= test_df.rel.values

# 3) Predicciones baseline
global_mean = train_df.rating.mean()
user_mean   = train_df.groupby('user_id').rating.mean()

# arrays de predicción
y_pred_global = np.full_like(y_true, fill_value=global_mean, dtype=float)
y_pred_user   = [
    user_mean.loc[row.user_id] if row.user_id in user_mean.index else global_mean
    for _, row in test_df.iterrows()
]

# 4) Regresión
mae_global, rmse_global = mean_absolute_error(y_true, y_pred_global), \
                          np.sqrt(mean_squared_error(y_true, y_pred_global))
mae_user,   rmse_user   = mean_absolute_error(y_true, y_pred_user), \
                          np.sqrt(mean_squared_error(y_true, y_pred_user))

# 5) Clasificación binaria (umbral 4)
y_pred_global_bin = (np.array(y_pred_global) >= 4).astype(int)
y_pred_user_bin   = (np.array(y_pred_user)   >= 4).astype(int)

prec_g = precision_score(y_true_bin, y_pred_global_bin, zero_division=0)
rec_g  = recall_score   (y_true_bin, y_pred_global_bin, zero_division=0)
f1_g   = f1_score       (y_true_bin, y_pred_global_bin, zero_division=0)

prec_u = precision_score(y_true_bin, y_pred_user_bin, zero_division=0)
rec_u  = recall_score   (y_true_bin, y_pred_user_bin, zero_division=0)
f1_u   = f1_score       (y_true_bin, y_pred_user_bin, zero_division=0)

# 6) nDCG@5
# construimos matrices completas de predicción para ranking
pred_global_mat = np.full((NUM_USERS, NUM_ITEMS), global_mean)
pred_user_mat   = np.zeros((NUM_USERS, NUM_ITEMS))
for _, row in test_df.iterrows():
    u = int(row.user_id)-1
    i = int(row.book_id)-1
    pred_user_mat[u,i] = user_mean.loc[row.user_id]

ndcg_g = get_ndcg(pred_global_mat, test_df, K=5, rel_col='rel')
ndcg_u = get_ndcg(pred_user_mat,   test_df, K=5, rel_col='rel')

# 7) Mostramos en DataFrame
results = pd.DataFrame({
    'Baseline': ['Global Mean', 'User Mean'],
    'RMSE':     [rmse_global, rmse_user],
    'MAE':      [mae_global, mae_user],
    'Precision':[prec_g, prec_u],
    'Recall':   [rec_g,  rec_u],
    'F1':       [f1_g,   f1_u],
    'nDCG@5':   [ndcg_g, ndcg_u]
})

print('Resultados de Baseline:\n', results)
